TODO: "Airs": to be aired like M2020

In [ ]:
import bs4 
from urllib.request import urlopen as uReq
from urllib.request import Request
from bs4 import BeautifulSoup as soup
import re
from math import ceil

In [ ]:
def read_page(my_url):
  uClient = uReq(Request(my_url, headers={'User-Agent': 'Mozilla/5.0'}))
  page_html = uClient.read()
  uClient.close()
  page_soup = soup(page_html, "html.parser")
  return page_soup

In [ ]:
my_url =  "https://mydramalist.com/search?adv=titles&ty=68&co=3&re=2019,2020&st=3&so=top"

In [ ]:
page_soup = read_page(my_url)
def find_total_result(page_soup = page_soup):
  total_res = page_soup.findAll("p", {"class": "m-b-sm pull-right"})
  total = int(re.search(r'\d+', total_res[0].text.strip()).group())
  return total

In [ ]:
def find_all_titles(page_soup = page_soup):
  preview_container = page_soup.findAll("div", {"class": "col-lg-8 col-md-8"})
  previews = preview_container[0].findAll("div", {"class": "row"})
  query = []
  for prev in previews:
    query.append(prev.a["href"])
  return query

Get all previews first then get details of each drama

In [ ]:
def find_title(page_soup = page_soup):
  try:
   title = page_soup.h1.a["title"]
  except:
    title = ""
  return title

def find_description(page_soup = page_soup):
  try:
    description = page_soup.findAll("div", {"class": "show-synopsis"})[0]
    description = description.p.text
    p = re.compile('(\n\(Source: .*\).*|Edit Translation)')
    description = p.sub('', description)
  except:
    description = ""
  return description

In [ ]:
def find_meta1(page_soup = page_soup):
  details = page_soup.findAll("div", {"class": "show-detailsxss"})[0]
  detail_dict = dict.fromkeys(["Genres" , "Screenwriter", "Director", "Tags", ])

  for name, _ in detail_dict.items():
    try:
      parent = details.find("b", text="{}:".format(name)).parent
      detail_html = parent.findAll("a")
      for i in range(len(detail_html)):
        detail_html[i] = detail_html[i].text
      detail_dict[name] = detail_html
    except:
      detail_dict[name] = []
  if(len(detail_dict['Tags'])>0):
    detail_dict['Tags'].pop()
  return detail_dict

In [ ]:
def find_meta2(page_soup = page_soup):
  detail_dict = dict.fromkeys(["Episodes", "Aired", "Aired On", "Original Network", "Score", "Ranked", "Popularity", "Watchers"])
  ul = page_soup.findAll('ul', {"class": "list m-a-0 hidden-md-up"})[0]
  for name, _ in detail_dict.items():
    try:
      detail = ul.find('b', text = "{}:".format(name)).parent
      detail = detail.text
      p = re.compile('(.*: )')
      detail = p.sub('', detail)
      detail_dict[name] = detail
    except:
      detail_dict[name] = ""
  return detail_dict

In [ ]:
def find_casts(page_soup = page_soup):
  actors = page_soup.findAll("div", { "class": "col-xs-8 col-sm-7 p-a-0"})
  casts = list()
  for actor in actors:
    try:
      role = actor.findAll("small", {"class": "text-muted"})[0]
      role = role.text
      if(role == "Main Role"):
        casts.append(actor.a['title'])
    except:
      continue
  return casts

In [ ]:
def preprocess_meta(meta):
  #popularity
  if(len(meta["Popularity"])>0):
    p = re.compile('#')
    meta["Popularity"] = p.sub('', meta["Popularity"])
    try:
      meta["Popularity"] =int(meta["Popularity"])
    except:
      pass

  #rank
  if(len(meta["Ranked"])>0):
    p = re.compile('#')
    meta["Ranked"] = p.sub('', meta["Ranked"])
    try:
      meta["Ranked"] =int(meta["Ranked"])
    except:
      pass

  #episodes
  if(len(meta["Episodes"])>0):
      meta["Episodes"] =int(meta["Episodes"])

  #score 
  if(len(meta["Score"])>0):
    try:
      results = re.search('(\d|.+|N/A) \(scored by (\d+(?:,\d+)?) users?\)', meta["Score"])
      user_rated = results.group(2)
      user_rated = user_rated.replace(',', '')
      user_rated = int(user_rated)
      if(user_rated == 0):
        score = 0
      else:
        score = float(results.group(1))
      meta["User_rated"] = user_rated
      meta["Score"] = score
    except:
      pass

  #watchers
  if(len(meta["Watchers"])>0):
    meta["Watchers"] = meta["Watchers"].replace(',', '')
    meta["Watchers"] =int(meta["Watchers"])

  #networks
  if (len(meta["Original Network"])>0):
    try:
      networks = meta["Original Network"].split()
      networks = list(set(networks))
      networks = (',').join(networks)
      meta["Original Network"] = networks
    except:
      pass
  
  return meta

In [ ]:
url = 'https://mydramalist.com/36269-doctor-playbook'
#page_soup = individual drama detail page
def get_kdrama(url):
  meta = dict()
  page_soup = read_page(url)
  title = find_title(page_soup)
  meta.update({"Title": title})
  description = find_description(page_soup)
  meta.update({"Description": description})
  meta_dict = find_meta1(page_soup)
  meta.update(meta_dict)
  meta_dict = find_meta2(page_soup)
  meta.update(meta_dict)
  casts = find_casts(page_soup)
  meta.update({"Casts": casts})
  return meta
  

In [ ]:
#Scripts
# sample url for drama details"https://mydramalist.com/36269-doctor-playbook"
base_url = "https://mydramalist.com"
my_url =  "https://mydramalist.com/search?adv=titles&ty=68&co=3&re=2016,2020&st=3&so=top"

kdrama_raw_list = list() #list of dict
kdrama_list = list() #list of dict

page_soup = read_page(my_url)
total_result = find_total_result(page_soup)
title_ids = find_all_titles(page_soup)
total_pages = ceil(total_result/len(title_ids))

for i in range(total_pages-1):
  target_url = my_url+"&page={}".format(i+2)
  page_soup = read_page(target_url)
  title_ids.extend(find_all_titles(page_soup))

for title in title_ids:
  kdrama_raw = get_kdrama(base_url + title)
  kdrama_raw_list.append(kdrama_raw)

kdrama_copy = kdrama_raw_list.copy()

for kdrama in kdrama_raw_list:
  kdrama = preprocess_meta(kdrama)
  kdrama_list.append(kdrama)

In [ ]:
import xlwt 
from xlwt import Workbook 
wb = Workbook() 
sheet1 = wb.add_sheet('Sheet 1') 

headers = ["Title", "Description", "Genres", "Director", "Screenwriter", "Tags", "Episodes", 
           "Aired", "Aired On", "Original Network", "Score", "User_rated", "Watchers", "Ranked", "Popularity", "Casts"]

for i in range(len(headers)):
  sheet1.write(0, i, headers[i])

for row_idx, kdrama in enumerate(kdrama_list):
  for col_idx in range(len(headers)):
    try:
      data = kdrama[headers[col_idx]]
    except:
      print(kdrama["Title"])
    if(type(data) == list or type(data) == bs4.element.ResultSet):
      data = str(data)
    sheet1.write(row_idx+1, col_idx, data)

wb.save('xlwt example.xls') 

In [ ]:
# class kdrama:
#   def __init__(self, title, description, genres, director, screen_writer, tags, episodes, 
#                aired, aired_on, network, rating, user_rated, total_watchers, rank, popularity, cast):
#     self.title = title,#
#     self.description = description,#
#     self.genres = genres,#
#     self.director = director,#
#     self.screen_writer = screen_writer,#
#     self.tags = tags,#
#     self.episodes = episodes,#
#     self.aired = aired,#
#     self.aired_on = aired_on,#
#     self.network = network,#
#     self.rating = rating,#
#     self.user_rated = user_rated, 
#     self.total_watchers = total_watchers,
#     self.rank = rank#
#     self.popularity = popularity#
#     self.cast = classmethod

#   def __str__(self):
